# **Signa_Lab ITESO:** Generador de *Embbeddings*

## **Cuaderno 01:** Limpieza y depuración de texto para procesar *embeddings* desde cuerpos de texto.

Cuaderno de código abierto diseñado para importar cualquier cuerpo de texto separado por filas, en formato CSV o Excel, limpiarlo (*stopwords*, URLs) y depurarlo desde [diccionarios personalizados](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=drive_link) (opcional) para optimizar su posterior procesamiento para generar incrustaciones de texto (*embeddings*) de cada fila, con ayuda de modelos de lenguaje de la librería [sentence-transformers](https://www.sbert.net/), alojados en repositorios de [HuggingFace](https://huggingface.co/sentence-transformers) (en la nube) o descargados localmente.

## 1. Importar librerías y archivos de datos a depurar

**Instalar librerías necesarias**

In [ ]:
# Instalar librerías de Python necesarias

!pip install pandas
!pip install nltk
!pip install difflib
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install numpy
!pip install plotly
!pip install time
!pip install tqdm
!pip install operator

**Importar librerías** necesarias:

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
import nltk
import re
import sys
import re
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.stats import gaussian_kde
import numpy as np
import plotly.express as px
import time
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import math
import operator

**Importar archivos de datos** con registros recibidos y diccionario para depuración por descarte:

In [ ]:
# PARA UN SOLO ARCHIVO (individual):
# Importar un solo archivo. Especificar ruta de archivo y extensión correspondiente a su formato:
ruta = "./nombre-archivo.csv" # archivo CSV
# ruta = "./nombre-archivo.xlsx" # archivo Excel

# Crear data frame con datos importados
df = pd.read_csv(ruta) # desde archivo CSV
# df = pd.read_excel(ruta) # desde archivo Excel


# PARA MÚLTIPLES ARCHIVOS (concatenar):
# Importar múltiples archivos para concatenar. Especificar rutas y extensiones:
# ruta1 = "./nombre-archivo1.csv" # importar CSV
# ruta2 = "./nombre-archivo2.csv" # importar CSV

# ruta1 = "./nombre-archivo1.xlsx" # importar Excel
# ruta2 = "./nombre-archivo1.xlsx" # importar Excel

# Crear data frames por cada archivo de datos importados:

# desde CSV:
# df1 = pd.read_csv(ruta1)
# df2 = pd.read_csv(ruta2)

# desde Excel:
# df1 = pd.read_excel(ruta1) # desde archivo Excel
# df2 = pd.read_excel(ruta2) # desde archivo Excel


# NOMBRE DEL PROYECTO
# Especificar nombre del proyecto, que se usará para nombrar los archivos de datos a generar y descargar:
nombreProyecto = 'nombre-proyecto'

Revisar **número de registros importados**

In [ ]:
# Para un solo archivo importado:

# Revisar el número de filas y columnas en archivo:
print("Filas y columnas en archivo importado:")
df.shape

In [ ]:
# Para múltiples archivos importados:

# Revisar el número de filas y columnas en archivo 1
# print("Filas y columnas en archivo 1:")
# df1.shape

In [ ]:
# Para múltiples archivos importados:

# Revisar el número de filas y columnas en archivo 2
# print("Filas y columnas en archivo 2:")
# df2.shape

In [ ]:
# Para múltiples archivos importados:

# Concatenar en una tabla todos los archivos importados y revisar el número de filas y columnas totales
# df = pd.concat([df1, df2],axis=0,ignore_index=True)
# df.shape

In [ ]:
# Crear copia de trabajo de la tabla de registros importados
dfTest = df.copy()

In [ ]:
# Previsualizar tabla con los registros importados
dfTest

In [ ]:
# Exportar archivo CSV con tabla de registros importados (y concatenados, en el caso de múltiples archivos)
dfTest.to_csv(f"{nombreProyecto.split('.')[0]}_registros-importados.csv")

---

## 2. Limpieza y depuración de registros importados

Generar identificadores únicos (IDs) por registro:

In [ ]:
# Definir función para asignar IDs únicos a cada fila en el data frame indicado como parámetro, comenzando desde '1000001'.
def assign_unique_ids(df):
    # Inicializar contador para IDs
    id_counter = 1000001

    # Crear copia de data frame original
    df_copy = df.copy()

    # Iterar a través de las filas del data frame
    for index, _ in enumerate(df_copy.index):
        # Dar formato a ID con ceros adicionales e incorporarlo al data frame
        formatted_id = str(id_counter).zfill(7)  # Se asegura de que sea un ID de 7 dígitos, agregando ceros cuando sea necesario
        df_copy.loc[index, 'id'] = formatted_id

        # Incrementar el contador del ID para la siguiente iteración
        id_counter += 1

    return df_copy

In [ ]:
# Ejecutar función para asignar IDs a cada registro
if __name__ == "__main__":
    # Invocar la función con data frame de trabajo
    df_ids = assign_unique_ids(dfTest)

df_ids

In [ ]:
# Sobreescribir data frame con nueva tabla con IDs generados
df = df_ids
df.head()

In [ ]:
# Revisar el número de filas y columnas en tabla de registros con IDs generados
df.shape

### Limpieza de texto sin aporte semántico:

In [ ]:
# Definir función para limpiar usuarios, hashtags y URLs
def limpiar_texto(texto):
  # Eliminar usuarios (opcional, comentar siguiente línea para omitirlo)
  texto = re.sub(r"(?<!\w)@(\w+)(?!\w)", "", texto)

  # Eliminar hashtags (opcional, comentar siguiente línea para omitirlo)
  # texto = re.sub(r"(?<!\w)#(\w+)(?!\w)", "", texto)

  # Eliminar URLs (opcional, comentar siguiente línea para omitirlo)
  texto = re.sub(r"(http|https|ftp)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", "", texto)
  texto = texto.lstrip(". ")

  return texto.strip()

**Ejecutar funciones y agregar nueva columna con registros con texto limpio:**

In [ ]:
# Definir función para agregar nueva columna con texto limpio (clean_text)
def agregarCleanTextADf(df, colText):
    dfW = df.copy()
    dfW["clean_text"] = None

    for index, row in dfW.iterrows():
        text = str(row[colText])
        cleaned_text = limpiar_texto(text)
        dfW.at[index, "clean_text"] = cleaned_text

    return dfW

In [ ]:
# Especificar nombre de columna con texto para limpiar y procesar embeddings
text_column = "nombre-columna-texto"

# Ejecutar limpieza de usuarios, hashtags y URLs
dfCleanText = agregarCleanTextADf(df, text_column)

In [ ]:
# Previsualizar tabla con nueva columna de datos limpios (clean_text)
dfCleanText.head()

### Eliminar palabras vacías (*stop words*):

Descargar dependencias de librería NLTK para **limpieza de palabras vacías (*stop words*):**

In [ ]:
# Descargar las stop words en español e inglés
nltk.download('punkt')
nltk.download('stopwords')

# Lista de stopwords en español
stopwords_es = nltk.corpus.stopwords.words('spanish')
# stopwords_en = nltk.corpus.stopwords.words('english')

**Función para eliminar palabras vacías (*stop words*)**

In [ ]:
# Definir función para eliminar palabras vacías (stopwords) y signos
def delete_stopwords(texto):

  # Tokenizar el texto
  tokens = nltk.word_tokenize(texto)

  # Eliminar signos de puntuación
  tokens = [token for token in tokens if token.isalnum()]

  # Eliminar stop words
  tokens = [token for token in tokens if token not in stopwords_es]

  # Convertir la lista de tokens a un string
  texto_limpio = " ".join(tokens)

  return texto_limpio.strip()

Ejecutar funciones y **agregar columna con texto sin *stop words* (sem_text):**

In [ ]:
# Definir función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
def agregarSemTextADf(df, colText):
    dfW = df.copy()
    dfW["sem_text"] = None

    for index, row in dfW.iterrows():
        text = row[colText]
        cleaned_text = delete_stopwords(text)
        dfW.at[index, "sem_text"] = cleaned_text

    return dfW

In [ ]:
# Ejecutar función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
dfFinal = agregarSemTextADf(dfCleanText, "clean_text")

In [ ]:
# Previsualizar tabla de datos con nueva columna de registros con texto sin palabras vacías (sem_text)
dfFinal.head()

In [ ]:
# Revisar el número de filas y columnas en tabla de registros con texto sin palabras vacías (sem_text)
dfFinal.shape

In [ ]:
# Exportar archivo CSV con tabla completa de registros importados con IDs y texto sin palabras vacías (sem_text)
dfFinal.to_csv(f"{nombreProyecto.split('.')[0]}_registros-semtext.csv")

## 3. Depuración de registros desde diccionarios personalizados con términos de descarte







A partir de diccionarios personalizados con términos de descarte, depurar registros que contengan alguno de ellos.

El diccionario debe subirse en formato CSV y contener, al menos, los siguientes campos:

| palabra | tipo | categoría | diccionario |
|---------|------|-----------|-------------|
|         |      |           |             |
|         |      |           |             |
|         |      |           |             |

Ejemplo:

| palabra | tipo   | categoría   | diccionario        |
|---------|--------|-------------|--------------------|
| idiota  | ofensa | humillación | ofensa-humillación |
| zorra   | ofensa | género      | ofensa-género      |
|         |        |             |                    |

Puedes [encontrar aquí una copia del diccionario](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=sharing) con la estructura requerida, en formato CSV, para descargar, llenar (opcionalmente) e incorporar a este cuaderno de código.

**Nota:** La versión actual de este código requiere importar y usar un diccionario en CSV, aún cuando esté vacío. Si no quieres depurar registros por términos de descarte, puedes solo descargar el diccionario de prueba y subirlo sin agregar términos, solo con los nombres de las columnas indicadas anteriormente.

In [ ]:
# Cargar diccionario con términos de descarte para depuración de términos específicos:
rutaDicc = './diccionario-prueba_01.csv'

# rutaDicc = 'diccionario-personalizado.csv'
dfDescarte = pd.read_csv(rutaDicc)

**Previsualizar diccionario de descarte importado:**

In [ ]:
# Previsualizar tabla de diccionario con términos de descarte
dfDescarte.head()

In [ ]:
# Revisar el número de filas y columnas en diccionario importado
dfDescarte.shape

In [ ]:
# Definir función para cotejar cada registro con términos de descarte, eliminar las coincidencias y agregar su razonamiento en nueva columna
def filtrar_registros(df_registros, df_terminos_descarte, colTexto):
    # Definir los terminos proscritos que ameritan eliminar el registro
    terminos = df_terminos_descarte["palabra"].tolist()
    # Compilar expresiones regulares una sola vez
    expresiones_regex = [re.compile(r"(?<!\S)?(?:\s|[.,;:?!¡¿]){}(?:\s|[.,;:?!¡¿])?(?!\S)".format(re.escape(termino)), re.IGNORECASE) for termino in terminos]

    # Columnas adicionales
    df_registros_filtrados = df_registros.copy()
    df_registros_filtrados["contiene_termino_descarte"] = False
    df_registros_filtrados["razon_eliminacion"] = ""

    # Filtrar registros
    for i in range(df_registros_filtrados.shape[0]):
        texto = str(df_registros_filtrados.loc[i, colTexto]).lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

        # Buscar coincidencias con expresiones regulares
        for expresion, razon in zip(expresiones_regex, df_terminos_descarte["categoría"]):
            coincidencias = expresion.findall(f" {texto} ")
            if coincidencias:
                df_registros_filtrados.loc[i, "contiene_termino_descarte"] = True
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Presencia de términos relacionados a {razon}"
                break

    # Eliminar registros que contienen términos de descarte
    df_registros_eliminados = df_registros_filtrados[df_registros_filtrados["contiene_termino_descarte"]]
    df_registros_filtrados = df_registros_filtrados[~df_registros_filtrados["contiene_termino_descarte"]]

    # Eliminar columnas auxiliares
    del df_registros_filtrados["contiene_termino_descarte"]
    del df_registros_filtrados["razon_eliminacion"]
    del df_registros_eliminados["contiene_termino_descarte"]

    df_registros_filtrados = df_registros_filtrados.reset_index(drop=True)

    return df_registros_filtrados, df_registros_eliminados


In [ ]:
# Ejecutar depuración de registros por palabras de descarte en diccionario
datasetSinTerminosProhibidos, datasetRegistrosEliminadosPorDescarte = filtrar_registros(dfFinal, dfDescarte,text_column)

In [ ]:
# Revisar el número de filas y columnas en tabla de registros depurados por palabras de descarte en diccionario
datasetSinTerminosProhibidos.shape

**Tamaño dataset con términos descartados**

In [ ]:
# Revisar el número de filas y columnas en tabla de registros eliminados por palabras de descarte en diccionario
datasetRegistrosEliminadosPorDescarte.shape

In [ ]:
datasetRegistrosEliminadosPorDescarte.head()

In [ ]:
#Verificar eliminación de casos específicos (opcional)
verificar_termino = ""

_count = 0
for i in datasetRegistrosEliminadosPorDescarte['clean_text']:
    words = i.split()
    if verificar_termino in words:
        _count += 1
_count

**Eliminar registros repetidas:**

Eliminar aquellos registros que contengan una similitud en su redacción mayor a un umbral establecido (por default asignado al 100%), para así buscar eliminar registros con una repetición exacta.


In [ ]:
# Definir función para calcular la similitud entre dos listas de palabras
def Similarity_Score(list1, list2):
    # Inicializar contadores para coincidencias y longitud total
    matches = 0
    total_length = 0

    # Iterar sobre las listas hasta el tamaño de la lista más corta
    for i in range(min(len(list1), len(list2))):
        # Si las palabras en las mismas posiciones coinciden, incrementar el contador de coincidencias
        if list1[i] == list2[i]:
            matches += 1
        # Incrementar el contador de longitud total
        total_length += 1

    # Para las posiciones adicionales en la lista más larga, incrementar el contador de longitud total
    for i in range(min(len(list1), len(list2)), max(len(list1), len(list2))):
        total_length += 1

    # Calcular el ratio de coincidencias como la proporción de coincidencias sobre la longitud total
    ratio = matches / total_length

    return ratio

**Definir función para identificar registros repetidos:**

In [ ]:
def remove_duplicates_with_threshold(df, column, threshold):
    global similarity_score
    print("Se actualizó")
    indices_to_remove = set()
    question_frequency = defaultdict(int) # Diccionario para almacenar la frecuencia de registros similares
    discarded_info = defaultdict(list) # Diccionario para almacenar información de registros descartados

    # Crear índice invertido para las palabras en los registros
    inverted_index = defaultdict(set)
    for i, question in enumerate(df[column]):
        words = set(question.split())
        for word in words:
            inverted_index[word].add(i)

    print(f"{len(df[column])} registros en total")
    for i, question in enumerate(df[column]):
        if i not in indices_to_remove:
            similar_questions_count = 1 # Contador de registros similares para la fila actual
            words = set(question.split())
            relevant_indices = set()
            for word in words:
                relevant_indices |= inverted_index[word]

            for j in relevant_indices:
                if j != i and j not in indices_to_remove:
                    registroSinAcentos = question.replace('á', 'a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
                    registroSinAcentosEnLista = registroSinAcentos.split(" ")
                    registroAComparar = df[column][j]
                    registroACompararSinAcentos = registroAComparar.replace('á', 'a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
                    registroACompararEnLista = registroACompararSinAcentos.split(" ")

                    score = Similarity_Score(list(registroSinAcentosEnLista), list(registroACompararEnLista))
                    if score >= threshold:
                        indices_to_remove.add(j)
                        similar_questions_count += 1
                        # Almacenar información de la registro descartado
                        discarded_info[j].append({'original_index': df['id'][i], 'similarity_score': score})
            question_frequency[i] = similar_questions_count # Almacenar la frecuencia de registros similares para la fila actual

        if i % 1000 == 0:
            print(f"Van {i} registros revisados...")

    # Eliminar los registros duplicados después de completar el bucle
    filtered_df = df.drop(indices_to_remove).reset_index(drop=True)

    # Crear DataFrame con registros duplicados
    df_removed_duplicates = df.iloc[list(indices_to_remove)]

    # Agregar información de registros descartados al DataFrame de registros descartados
    id_match = []
    similarity_score = []

    # Iterar sobre índice de DataFrame
    for index in df_removed_duplicates.index:
        # Revisar si el índice se encuentra en discarded_info
        if index in discarded_info:
            # Por cada índice, toma el primer elemento de 'original_index' y 'similarity_score'
            id_match.append(discarded_info[index][0]['original_index'])
            similarity_score.append(discarded_info[index][0]['similarity_score'])
        else:
            # Si el índice no está en discarded_info, agregar el valor por default None
            id_match.append(None)
            similarity_score.append(None)

    df_removed_duplicates['id_match'] = id_match
    df_removed_duplicates['similarity_score'] = similarity_score

    filtered_df['question_frequency_count'] = filtered_df['id'].apply(
        lambda x: len(df_removed_duplicates[df_removed_duplicates['id_match'] == x]) + 1)

    return filtered_df, df_removed_duplicates

Ejecutar función para **eliminar registros duplicados**

In [ ]:
# Establecer umbral de similitud (porcentaje)
threshold = 1 # Se eliminan registros que sean 100% similares en su redacción a alguno ya registrado
# Ejecutar elminación de registros repetidos
df_filtered, df_removed_duplicates = remove_duplicates_with_threshold(datasetSinTerminosProhibidos, 'sem_text', threshold)#.head(1000)

In [ ]:
# Previsualizar tabla de población de registros depurados con la frecuencia de aparición de cada registro respecto a otros registros
df_filtered.head()

In [ ]:
# Previsualizar tabla de registros eliminados por repetición
df_removed_duplicates.head()

In [ ]:
# Definir función para agregar la razón de eliminación por repetidos
def agregar_razon_eliminacion(df_removed, razon):
    df_removed['razon_eliminacion'] = razon
    return df_removed

df_removed_duplicates = agregar_razon_eliminacion(df_removed_duplicates, 'Redacción repetida respecto a otro registro')

In [ ]:
# Previsualizar tabla de registros eliminados por repetición con razón de eliminación
df_removed_duplicates

In [ ]:
# Revisar el número de filas y columnas en tabla de registros eliminados por repetición
df_removed_duplicates.shape

**Concatenar tabla con filas eliminadas por términos de descarte en diccionario y por repeticiones:**

In [ ]:
# Ejecutar concatenación de filas eliminadas por términos en diccionario y repeticiones
datasetRegistrosEliminados = pd.concat([datasetRegistrosEliminadosPorDescarte, df_removed_duplicates], axis=0)

In [ ]:
# Previsualizar tabla de registros eliminados por términos en diccionario y repeticiones con razón de eliminación
datasetRegistrosEliminados

In [ ]:
# Previsualizar tabla de población de registros depurados
df_filtered

## 4. Revisar y exportar datos con registros depurados y eliminados

In [ ]:
# Revisar el número de filas y columnas en tabla de población de registros depurados
df_filtered.shape

In [ ]:
# Revisar el número de filas y columnas en tabla concatenada de registros eliminados por términos de descarte en diccionarios y repeticiones
datasetRegistrosEliminados.shape

**Exportar archivo de datos (en formato CSV) de población de registros depurados a utilizar:**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con población de registros depurados
df_filtered.to_csv(f"{nombreProyecto.split('.')[0]}_PoblacionRegistrosDepurados.csv")

print(f"¡{nombreProyecto.split('.')[0]}_PoblacionRegistrosDepurados.csv descargado!")

**Exportar archivo de datos (en formato CSV) de registros eliminados por términos de descarte o repeticiones, con su razonamiento correspondiente:**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con registros eliminados por términos de descarte en diccionarios y repeticiones
datasetRegistrosEliminados.to_csv(f"{nombreProyecto.split('.')[0]}_RegistrosDescartados.csv")

print(f"¡{nombreProyecto.split('.')[0]}_RegistrosDescartados.csv descargado!")

## 5. Referencias

*   Bird, Steven, Edward Loper & Ewan Klein (2009).
Natural Language Processing with Python.  O'Reilly Media Inc.
*   Guzmán Falcón, E. (2018). Detección de lenguaje ofensivo en Twitter basada en expansión automática de lexicones (Tesis de Maestría). Instituto Nacional de Astrofísica, Óptica y Electrónica. Recuperado de https://inaoe.repositorioinstitucional.mx/jspui/bitstream/1009/1722/1/GuzmanFE.pdf
* Kiss, T., & Strunk, J. (2006). Unsupervised Multilingual Sentence Boundary Detection. Computational Linguistics, 32(4), 485-525. https://doi.org/10.1162/coli.2006.32.4.485

## 6. Créditos

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Javier de la Torre Silva, José Luis Almendarez González y Diego Arredondo Ortiz

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Mayo, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.


---